# Age-sex matched & IOP-removed datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import csv
import miceforest as mf
from tqdm import tqdm



import joblib
from joblib import load

import sklearn
from sklearn.preprocessing import MinMaxScaler

import scipy.stats as stats

import importlib

import optuna
from optuna import pruners
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna.samplers import TPESampler

import imblearn
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours
from imblearn.combine import SMOTEENN

import sklearn

import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import importlib
from joblib import dump, load
import os
import math
from functools import reduce

import torch
import torch.nn as nn
from torch.nn import ReLU
import random

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE, RFECV, SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression


import sys
sys.path.append('../')

import model_util
importlib.reload(model_util)
from model_util import get_scoring_metrics

#import optuna_util
#importlib.reload(optuna_util)
#from optuna_util import run_optuna_studies

import feature_sets
importlib.reload(feature_sets)

#import neural_net
#importlib.reload(neural_net)
#from neural_net import MLP

<module 'feature_sets' from '/home/eake191/resmed202100066-Glaucoma_PRS/emma_summer2023/honours/feature_sets.py'>

<br>

# Age-sex matched dataset


#### loading data

In [13]:
merged_df1 = pd.read_csv('incidence_glaucoma.csv')
merged_df2 = pd.read_pickle('data/derived/derived_cols_merged.pkl')
merged_df = pd.merge(merged_df1, merged_df2, on='f.eid')
merged_df = merged_df[merged_df['IOP subcohort'] == 1]
merged_df

,f.eid,glaucoma_control,diagnosis_source,age_diagnosed,date_diagnosed,date_attending_assessment_center,time_to_diagnosis,tte_3year,tte_5year,tte_10year,...,Normal sleep duration,Insomnia frequency,Snoring,Daytime sleeping frequency,Exclusion,Glaucoma (prevalent D|TD),IOP subcohort,training_test_split_90_10,Exercise (summed MET minutes per week),training_test_split_80_20
5,1000060,Control,NaN,NaN,NaN,2010-06-11,NaN,Control,Control,Control,...,1.0,1.0,1.0,0.0,0.0,Control,1,train,247.5,train
7,1000083,Control,NaN,NaN,NaN,2009-09-14,NaN,Control,Control,Control,...,1.0,1.0,0.0,0.0,0.0,Control,1,train,5454.0,train
9,1000102,Control,NaN,NaN,NaN,2010-05-10,NaN,Control,Control,Control,...,1.0,1.0,1.0,0.0,0.0,Control,1,train,NaN,train
20,1000217,Control,NaN,NaN,NaN,2010-01-09,NaN,Control,Control,Control,...,1.0,2.0,1.0,0.0,0.0,Control,1,train,462.0,train
22,1000234,Control,NaN,NaN,NaN,2010-06-11,NaN,Control,Control,Control,...,1.0,0.0,0.0,1.0,0.0,Control,1,train,198.0,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502406,6024070,Control,NaN,NaN,NaN,2010-05-17,NaN,Control,Control,Control,...,1.0,0.0,NaN,0.0,0.0,Control,1,train,3399.0,train
502409,6024107,Control,NaN,NaN,NaN,2010-04-26,NaN,Control,Control,Control,...,0.0,2.0,0.0,0.0,0.0,Control,1,train,1238.0,train
502410,6024119,Control,NaN,NaN,NaN,2009-10-15,NaN,Control,Control,Control,...,0.0,1.0,1.0,0.0,0.0,Control,1,train,NaN,train
502415,6024163,Incident,GP,61.0,2016-04-25,2009-10-03,6.55989,NaN,NaN,Glaucoma,...,1.0,1.0,0.0,0.0,0.0,Glaucoma,1,train,1935.0,train


In [19]:
df3 = pd.read_csv('william.csv')
data = {
    'ID': merged_df['f.eid'],
    'IOP_cohort_3': merged_df['tte_3year'],
    'IOP_cohort_5': merged_df['tte_5year'],
    'IOP_cohort_10': merged_df['tte_10year'] }

df = pd.DataFrame(data)
df2 = pd.merge(df, df3, on = 'ID', how = 'outer')
df2
df2.to_csv('william.csv', index=False)

#### matching function def

In [3]:
def match_cases_controls(cases_df, controls_df, match_features, ratio=10):
    """
    Match each case to R unique controls without replacement.
    
    Args:
        cases_df: DataFrame of cases
        controls_df: DataFrame of controls
        match_features: List of columns to match on
        ratio: Controls per case (default=10)
    
    Returns:
        matched_pairs: List of (case_id, [control_ids]) tuples
        matched_df: Combined DataFrame with match groups
    """
    # Reset indices for clean referencing
    cases_df = cases_df.reset_index(drop=True).copy()
    controls_df = controls_df.reset_index(drop=True).copy()
    
    # Convert features to numeric matrix
    X_cases = cases_df[match_features].values
    X_controls = controls_df[match_features].values
    
    # Initialize tracking
    used_controls = set()
    matched_pairs = []
    
    # Find ALL possible neighbors first
    nbrs = NearestNeighbors(n_neighbors=len(controls_df)).fit(X_controls)
    distances, all_indices = nbrs.kneighbors(X_cases)
    
    # Match each case to R unique controls
    for case_idx in range(len(cases_df)):
        case_id = cases_df.loc[case_idx, 'f.eid']
        candidates = all_indices[case_idx]
        matched_for_case = []
        
        for control_idx in candidates:
            if control_idx not in used_controls:
                control_id = controls_df.loc[control_idx, 'f.eid']
                matched_for_case.append(control_id)
                used_controls.add(control_idx)
                
                if len(matched_for_case) == ratio:
                    break
        
        matched_pairs.append((case_id, matched_for_case))
    
    # Create combined DataFrame
    all_case_ids = [p[0] for p in matched_pairs]
    all_control_ids = [cid for p in matched_pairs for cid in p[1]]
    
    matched_df = pd.concat([
        cases_df[cases_df['f.eid'].isin(all_case_ids)],
        controls_df[controls_df['f.eid'].isin(all_control_ids)]
    ])
    
    # Add match group identifiers
    match_map = {}
    for group_id, (case_id, control_ids) in enumerate(matched_pairs):
        match_map[case_id] = group_id
        for cid in control_ids:
            match_map[cid] = group_id
    
    matched_df['match_group'] = matched_df['f.eid'].map(match_map)
    
    # QC Reporting
    print(f"\n=== Matching Results ===")
    print(f"Cases matched: {len(all_case_ids)}")
    print(f"Controls matched: {len(all_control_ids)}")
    print(f"Average controls per case: {len(all_control_ids)/len(all_case_ids):.1f}")
    
    return matched_pairs, matched_df

<br>

## 3 year

<br>

### Case control matching

In [ ]:
cases = merged_df[merged_df['tte_3year'] == 'Glaucoma'].dropna(subset=["Age at initial assesement", "Sex"])
controls = merged_df[merged_df['tte_3year'] == 'Control'].dropna(subset=["Age at initial assesement", "Sex"])

pairs, matched_3year_tte = match_cases_controls(
    cases_df=cases,
    controls_df=controls,
    match_features=['Age at initial assesement', 'Sex'],
    ratio=10
)

In [ ]:
matched_controls = matched_3year_tte[matched_3year_tte['tte_3year'] == 'Control']


print(cases["Age at initial assesement"].mean())
print(matched_controls["Age at initial assesement"].mean())

print(cases["Sex"].mean())
print(matched_controls["Sex"].mean())

In [ ]:
len(matched_3year_tte['f.eid'].unique())

In [ ]:
matched_3year_tte = pd.concat([cases, matched_controls])
matched_3year_tte

<br>

### split into train/test (80/20)

In [ ]:
# 1. First, separate cases and controls
cases = matched_3year_tte[matched_3year_tte['tte_3year'] == 'Glaucoma']
controls = matched_3year_tte[matched_3year_tte['tte_3year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_3year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_3year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_3year_80_20_split'] = 'train'
test_df['tte_3year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
matched_3year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(matched_3year_tte['tte_3year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    matched_3year_tte['tte_3year_80_20_split'],
    matched_3year_tte['tte_3year'],
    normalize='index'
))

In [ ]:
matched_3year_tte.to_csv('matched_3year_tte.csv', index=False)

In [2]:
matched_3year_tte = pd.read_csv('matched_3year_tte.csv')

In [24]:
df = pd.read_csv('william.csv')

data = data = {
    'ID': matched_3year_tte['f.eid'],
    'matched3': matched_3year_tte['tte_3year'],
    'matched5': matched_3year_tte['tte_5year'],
    'matched10': matched_3year_tte['tte_10year'] }
df2 = pd.DataFrame(data)
df2

df3 = pd.merge(df2, df, on = 'ID', how = 'outer')
df3
df3.to_csv('william.csv', index=False)

### imputation

In [ ]:
odsl_feature_list = feature_sets.ODSL_features['feature'].values
odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
matched_3year_tte[odsl_categorical_features] = matched_3year_tte[odsl_categorical_features].astype('category')


X_train_3year_tte, y_train_3year_tte, X_test_3year_tte, y_test_3year_tte = model_util.get_train_test_datasets(
    matched_3year_tte, 
    'tte_3year_80_20_split', 
    'tte_3year', 
    odsl_feature_list)

In [ ]:
imputation_kernel_3year_tte = mf.ImputationKernel(
    X_train_3year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [ ]:
imputation_kernel_3year_tte.mice(
    verbose=True,
    iterations=15, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

In [ ]:
joblib.dump(imputation_kernel_3year_tte, './data/imputed/imputation_kernel_15iter_matched3year_tte.pkl')

<br>

### Save imputed data

In [ ]:
#imputation_kernel_3year_tte = load('./data/imputed/imputation_kernel_20iter_matched3year_tte.pkl')

In [ ]:
X_train_imputed_3year_tte = imputation_kernel_3year_tte.complete_data().reset_index(drop=True)
X_test_imputed_3year_tte = imputation_kernel_3year_tte.impute_new_data(X_test_3year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_matched3year_tte.pkl')
joblib.dump(X_test_imputed_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_matched3year_tte.pkl')

joblib.dump(y_train_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_matched3year_tte.pkl')
joblib.dump(y_test_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_matched3year_tte.pkl')

In [ ]:
X_merged_imputed_3year_tte = pd.concat((X_train_imputed_3year_tte, X_test_imputed_3year_tte), ignore_index=True)
y_merged_3year_tte = pd.concat((y_train_3year_tte, y_test_3year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_3year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_matched3year_tte.pkl')
joblib.dump(y_merged_3year_tte, './data/imputed/IOPsubcohort_y_merged_matched3year_tte.pkl')

In [ ]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_3year_tte)

X_train_scaled_3year_tte = pd.DataFrame(scaler.transform(X_train_imputed_3year_tte), columns=X_train_3year_tte.columns)
X_test_scaled_3year_tte = pd.DataFrame(scaler.transform(X_test_imputed_3year_tte), columns=X_test_3year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_matched3year_tte.pkl')
joblib.dump(X_test_scaled_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_matched3year_tte.pkl')

<br>

## 5 year

In [ ]:
# case control matching
cases = merged_df[merged_df['tte_5year'] == 'Glaucoma'].dropna(subset=["Age at initial assesement", "Sex"])
controls = merged_df[merged_df['tte_5year'] == 'Control'].dropna(subset=["Age at initial assesement", "Sex"])


pairs, matched_5year_tte = match_cases_controls(
    cases_df=cases,
    controls_df=controls,
    match_features=['Age at initial assesement', 'Sex'],
    ratio=10
)

In [ ]:
matched_controls = matched_5year_tte[matched_5year_tte['tte_3year'] == 'Control']


print(cases["Age at initial assesement"].mean())
print(matched_controls["Age at initial assesement"].mean())

print(cases["Sex"].mean())
print(matched_controls["Sex"].mean())

len(matched_5year_tte['f.eid'].unique())

In [ ]:
matched_5year_tte = pd.concat([cases, matched_controls])
matched_5year_tte

<br>

### split into train/test (80/20)

In [ ]:
# 1. First, separate cases and controls
cases = matched_5year_tte[matched_5year_tte['tte_5year'] == 'Glaucoma']
controls = matched_5year_tte[matched_5year_tte['tte_5year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_5year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_5year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_5year_80_20_split'] = 'train'
test_df['tte_5year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
matched_5year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(matched_5year_tte['tte_5year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    matched_5year_tte['tte_5year_80_20_split'],
    matched_5year_tte['tte_5year'],
    normalize='index'
))

In [ ]:
matched_5year_tte.to_csv('matched_5year_tte.csv', index=False)

In [25]:
# run if needed
matched_5year_tte = pd.read_csv('matched_5year_tte.csv')

In [36]:
df = pd.read_csv('william.csv')

data = data = {
    'ID': matched_5year_tte['f.eid'],
    'matched5': matched_5year_tte['tte_5year']}
df2 = pd.DataFrame(data)
df2

df3 = pd.merge(df2, df, on = 'ID', how = 'outer')
df3
df3.to_csv('william.csv', index=False)

<br>

### Imputation

In [ ]:
odsl_feature_list = feature_sets.ODSL_features['feature'].values
odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
matched_5year_tte[odsl_categorical_features] = matched_5year_tte[odsl_categorical_features].astype('category')


X_train_5year_tte, y_train_5year_tte, X_test_5year_tte, y_test_5year_tte = model_util.get_train_test_datasets(
    matched_5year_tte, 
    'tte_5year_80_20_split', 
    'tte_5year', 
    odsl_feature_list)

In [ ]:
imputation_kernel_5year_tte = mf.ImputationKernel(
    X_train_5year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [ ]:
imputation_kernel_5year_tte.mice(
    verbose=True,
    iterations=15, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

In [ ]:
joblib.dump(imputation_kernel_5year_tte, './data/imputed/imputation_kernel_20iter_matched15year_tte.pkl')

<br>

### save imputed data

In [ ]:
# run if needed
imputation_kernel_5year_tte = load('./data/imputed/imputation_kernel_20iter_matched5year_tte.pkl')

In [ ]:
X_train_imputed_5year_tte = imputation_kernel_5year_tte.complete_data().reset_index(drop=True)
X_test_imputed_5year_tte = imputation_kernel_5year_tte.impute_new_data(X_test_5year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_matched5year_tte.pkl')
joblib.dump(X_test_imputed_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_matched5year_tte.pkl')

joblib.dump(y_train_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_matched5year_tte.pkl')
joblib.dump(y_test_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_matched5year_tte.pkl')

In [ ]:
X_merged_imputed_5year_tte = pd.concat((X_train_imputed_5year_tte, X_test_imputed_5year_tte), ignore_index=True)
y_merged_5year_tte = pd.concat((y_train_5year_tte, y_test_5year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_5year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_matched5year_tte.pkl')
joblib.dump(y_merged_5year_tte, './data/imputed/IOPsubcohort_y_merged_matched5year_tte.pkl')

In [ ]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_5year_tte)

X_train_scaled_5year_tte = pd.DataFrame(scaler.transform(X_train_imputed_5year_tte), columns=X_train_5year_tte.columns)
X_test_scaled_5year_tte = pd.DataFrame(scaler.transform(X_test_imputed_5year_tte), columns=X_test_5year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_matched5year_tte.pkl')
joblib.dump(X_test_scaled_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_matched5year_tte.pkl')

<br>

## 10 year

In [ ]:
# old code where i split into chunks cause kernel kept dying
# Prepare cases and controls
cases = merged_df[merged_df['tte_10year'] == 'Glaucoma'].dropna(subset=["Age at initial assesement", "Sex"])
controls = merged_df[merged_df['tte_10year'] == 'Control'].dropna(subset=["Age at initial assesement", "Sex"])

# Initialize tracking
matched_controls = set()
matched_pairs = []
matched_data = []

# Process cases in chunks
case_chunks = np.array_split(cases, max(1, len(cases)//100))  # ~100 cases per chunk

for case_chunk in tqdm(case_chunks, desc="Processing case chunks"):
    # For each case in chunk, find 10 unique controls
    for _, case_row in case_chunk.iterrows():
        # Find potential matches (not already matched)
        potential_controls = controls[
            (~controls.index.isin(matched_controls)) &
            (controls["Sex"] == case_row["Sex"]) &
            (np.isclose(controls["Age at initial assesement"], case_row["Age at initial assesement"], atol=1))  # Age ±1 year
        ]
        
        # Take first 10 available matches
        selected_controls = potential_controls.head(10)
        
        if len(selected_controls) == 10:
            matched_controls.update(selected_controls.index)
            matched_pairs.append((case_row.name, selected_controls.index.tolist()))
            matched_data.extend([case_row] + [selected_controls.loc[idx] for idx in selected_controls.index])

# Create final matched dataset
matched_10year_tte = pd.DataFrame(matched_data).reset_index(drop=True)

# Verify counts
case_count = matched_10year_tte['tte_10year'].value_counts()['Glaucoma']
control_count = matched_10year_tte['tte_10year'].value_counts()['Control']
print(f"Cases: {case_count}, Controls: {control_count}, Ratio: {control_count/case_count:.1f}:1")

In [ ]:
cases = merged_df[merged_df['tte_10year'] == 'Glaucoma'].dropna(subset=["Age at initial assesement", "Sex"])
controls = merged_df[merged_df['tte_10year'] == 'Control'].dropna(subset=["Age at initial assesement", "Sex"])

pairs, matched_10year_tte = match_cases_controls(
    cases_df=cases,
    controls_df=controls,
    match_features=['Age at initial assesement', 'Sex'],
    ratio=10
)

In [ ]:
matched_controls = matched_10year_tte[matched_10year_tte['tte_10year'] == 'Control']


print(cases["Age at initial assesement"].mean())
print(matched_controls["Age at initial assesement"].mean())

print(cases["Sex"].mean())
print(matched_controls["Sex"].mean())

len(matched_10year_tte['f.eid'].unique())

In [ ]:
matched_10year_tte = pd.concat([cases, matched_controls])
matched_10year_tte

<br>

### split into train/test (80/20 split)

In [ ]:
# separate cases and controls
cases = matched_10year_tte[matched_10year_tte['tte_10year'] == 'Glaucoma']
controls = matched_10year_tte[matched_10year_tte['tte_10year'] == 'Control']

# split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_10year']  
)

# for controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_10year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_10year_80_20_split'] = 'train'
test_df['tte_10year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
matched_10year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(matched_10year_tte['tte_10year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    matched_10year_tte['tte_10year_80_20_split'],
    matched_10year_tte['tte_10year'],
    normalize='index'
))

In [ ]:
matched_10year_tte.to_csv('matched_10year_tte.csv', index=False)

In [38]:
# run if needed
matched_10year_tte = pd.read_csv('matched_10year_tte.csv')

/tmp/ipykernel_1979962/2594143693.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  matched_10year_tte = pd.read_csv('matched_10year_tte.csv')


In [41]:
df = pd.read_csv('william.csv')

data = data = {
    'ID': matched_10year_tte['f.eid'],
    'matched10': matched_10year_tte['tte_10year']}
df2 = pd.DataFrame(data)
df2

df3 = pd.merge(df2, df, on = 'ID', how = 'outer')
df3
df3.to_csv('william.csv', index=False)

<br>

### imputation

In [ ]:
odsl_feature_list = feature_sets.ODSL_features['feature'].values
odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
matched_10year_tte[odsl_categorical_features] = matched_10year_tte[odsl_categorical_features].astype('category')


X_train_10year_tte, y_train_10year_tte, X_test_10year_tte, y_test_10year_tte = model_util.get_train_test_datasets(
    matched_10year_tte, 
    'tte_10year_80_20_split', 
    'tte_10year', 
    odsl_feature_list)

In [ ]:
imputation_kernel_10year_tte = mf.ImputationKernel(
    X_train_10year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [ ]:
imputation_kernel_10year_tte.mice(
    verbose=True,
    iterations=15, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

In [ ]:
joblib.dump(imputation_kernel_10year_tte, './data/imputed/imputation_kernel_15iter_matched10year_tte.pkl')

* from last night^ done up to here assuming the kernel didnt die (pls)

<br>

### save imputed data

In [ ]:
# run if needed
imputation_kernel_10year_tte = load('./data/imputed/imputation_kernel_20iter_matched10year_tte.pkl')

In [ ]:
X_train_imputed_10year_tte = imputation_kernel_10year_tte.complete_data().reset_index(drop=True)
X_test_imputed_10year_tte = imputation_kernel_10year_tte.impute_new_data(X_test_10year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_matched10year_tte.pkl')
joblib.dump(X_test_imputed_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_matched10year_tte.pkl')

joblib.dump(y_train_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_matched10year_tte.pkl')
joblib.dump(y_test_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_matched10year_tte.pkl')

In [ ]:
X_merged_imputed_10year_tte = pd.concat((X_train_imputed_10year_tte, X_test_imputed_10year_tte), ignore_index=True)
y_merged_10year_tte = pd.concat((y_train_10year_tte, y_test_10year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_10year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_matched10year_tte.pkl')
joblib.dump(y_merged_10year_tte, './data/imputed/IOPsubcohort_y_merged_matched10year_tte.pkl')

In [ ]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_10year_tte)

X_train_scaled_10year_tte = pd.DataFrame(scaler.transform(X_train_imputed_10year_tte), columns=X_train_10year_tte.columns)
X_test_scaled_10year_tte = pd.DataFrame(scaler.transform(X_test_imputed_10year_tte), columns=X_test_10year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_matched10year_tte.pkl')
joblib.dump(X_test_scaled_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_matched10year_tte.pkl')

<br>

<br>

# IOP-removed dataset

#### loading data

In [2]:
merged_df1 = pd.read_csv('/mnt/shared_folders/eResearch_glaucoma_project/emma_summer2023/honours/incidence_glaucoma.csv')
merged_df2 = pd.read_pickle('/mnt/shared_folders/eResearch_glaucoma_project/emma_summer2023/honours/data/derived/derived_cols_merged.pkl')
merged_df = pd.merge(merged_df1, merged_df2, on='f.eid')
merged_df.drop(['IOPcc_OS', 'IOPg_OD', 'IOPg', 'IOPg inter-eye difference', 'IOPg_OS_pretreatment',
                'IOPg_OD_pretreatment','IOPg pre-treatment','IOPg pre-treatment inter-eye difference',
                'IOPcc_OD','IOPcc','IOP_available','IOPcc inter-eye difference','on_IOP_medication_baseline',
                'IOPcc_OS_pretreatment','IOPcc_OD_pretreatment','IOPcc pre-treatment','IOPg_OS'], axis=1, inplace=True)

In [3]:
odsl_feature_list = feature_sets.ODSL_features['feature'].values
odsl_feature_list = odsl_feature_list[~np.isin(odsl_feature_list, ['IOPg pre-treatment inter-eye difference', 'IOPg pre-treatment'])]


odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
merged_df[odsl_categorical_features] = merged_df[odsl_categorical_features].astype('category')

<br>

### examining missing data

In [4]:
glaucoma_df_3year_tte = merged_df[merged_df['tte_3year'] == 'Glaucoma']
control_df_3year_tte = merged_df[merged_df['tte_3year'] == 'Control']

total_n = len(merged_df)
glaucoma_n = len(glaucoma_df_3year_tte)
control_n = len(control_df_3year_tte)

missing_feature_df_3year_tte = pd.DataFrame(columns=[
    'Feature',
    'N missing', # used for sorting
    'N missing (%)',
    'N missing, glaucoma (%)',
    'N missing, control (%)',
    'p',
])
missing_feature_df_3year_tte = missing_feature_df_3year_tte.set_index('Feature', drop=True)

for feature in odsl_feature_list:
    n_missing = merged_df[feature].isna().sum()
    n_missing_percent = (n_missing / total_n) * 100
    missing_feature_df_3year_tte.loc[feature, 'N missing'] = n_missing
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing (%)'] = f'{n_missing:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing (%)'] = f'{n_missing} ({n_missing_percent:0.2f}%)'

    # Glaucoma
    n_missing_glaucoma = glaucoma_df_3year_tte[feature].isna().sum()
    n_missing_percent = (n_missing_glaucoma / glaucoma_n) * 100
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing, glaucoma (%)'] = f'{n_missing_glaucoma:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing, glaucoma (%)'] = f'{n_missing_glaucoma} ({n_missing_percent:0.2f}%)'

    # Control
    n_missing_control = control_df_3year_tte[feature].isna().sum()
    n_missing_percent = (n_missing_control / control_n) * 100
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing, control (%)'] = f'{n_missing_control:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing, control (%)'] = f'{n_missing_control} ({n_missing_percent:0.2f}%)'

    # Chi-square

    if n_missing_glaucoma == 0 and n_missing_control == 0:
        continue

    # contignency tab
    ctb = pd.crosstab(index=merged_df[feature].isna() == True, columns=merged_df['tte_3year'])
    
    p_val = stats.chi2_contingency(ctb).pvalue
    if p_val < 0.001:
        p_val = '<0.001'
    else:
        p_val = f'{p_val:0.3f}'
    missing_feature_df_3year_tte.loc[feature, 'p'] = p_val

missing_feature_df_3year_tte = missing_feature_df_3year_tte.sort_values(by='N missing', axis=0, ascending=False)
missing_feature_df_3year_tte = missing_feature_df_3year_tte.drop(columns=['N missing'])
missing_feature_df_3year_tte.head(15)

,N missing (%),"N missing, glaucoma (%)","N missing, control (%)",p
Feature,,,,
Corneal hysteresis inter-eye difference,"391,541 (77.93%)","1,053 (79.65%)","378,591 (77.94%)",0.142
Spherical equivalent,"390,371 (77.70%)","1,053 (79.65%)","377,496 (77.71%)",0.097
Corneal hysteresis,"389,155 (77.46%)","1,045 (79.05%)","376,361 (77.48%)",0.183
Corneal resistance factor,"389,155 (77.46%)","1,045 (79.05%)","376,361 (77.48%)",0.183
Speech reception threshold,"336,894 (67.05%)",808 (61.12%),"325,884 (67.09%)",<0.001
Tinnitus frequency (self-reported),"334,158 (66.51%)",797 (60.29%),"323,298 (66.55%)",<0.001
Arterial stiffness index,"332,656 (66.21%)",790 (59.76%),"321,848 (66.25%)",<0.001
Private healthcare utilisation,"331,455 (65.97%)",787 (59.53%),"320,699 (66.02%)",<0.001
Exercise (summed MET minutes per week),"117,201 (23.33%)",346 (26.17%),"112,926 (23.25%)",0.013


In [4]:
removed_features = ['Corneal hysteresis inter-eye difference', 'Spherical equivalent', 'Corneal hysteresis',
                    'Corneal resistance factor',
                    'Speech reception threshold', 'Tinnitus frequency (self-reported)', 'Arterial stiffness index',
                    'Private healthcare utilisation', 'Exercise (summed MET minutes per week)', 'Total household income']

In [5]:
merged_df = merged_df.drop(removed_features, axis=1, inplace=False)


<br>

<br>

## 3 year

### split into train/test (80/20)

In [6]:
# 1. First, separate cases and controls
cases = merged_df[merged_df['tte_3year'] == 'Glaucoma']
controls = merged_df[merged_df['tte_3year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_3year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_3year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_3year_80_20_split'] = 'train'
test_df['tte_3year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOPremoved_3year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOPremoved_3year_tte['tte_3year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOPremoved_3year_tte['tte_3year_80_20_split'],
    IOPremoved_3year_tte['tte_3year'],
    normalize='index'
))

Final Split Proportions:
tte_3year_80_20_split
train    0.799998
test     0.200002
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_3year               Control  Glaucoma
tte_3year_80_20_split                    
test                   0.997280  0.002720
train                  0.997287  0.002713


In [8]:
IOPremoved_3year_tte.to_csv('IOPremoved_3year_tte.csv', index=False)

In [ ]:
# run if needed
IOPremoved_3year_tte = pd.read_csv('IOPremoved_10year_tte.csv')

### imputation

In [9]:
removed_features = ['Corneal hysteresis inter-eye difference', 'Spherical equivalent', 'Corneal hysteresis',
                    'Corneal resistance factor',
                    'Speech reception threshold', 'Tinnitus frequency (self-reported)', 'Arterial stiffness index',
                    'Private healthcare utilisation', 'Exercise (summed MET minutes per week)', 'Total household income']

In [10]:
odsl_feature_list = odsl_feature_list[~np.isin(odsl_feature_list, removed_features)]


odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
merged_df[odsl_categorical_features] = merged_df[odsl_categorical_features].astype('category')

In [11]:
X_train_3year_tte, y_train_3year_tte, X_test_3year_tte, y_test_3year_tte = model_util.get_train_test_datasets(
    IOPremoved_3year_tte, 
    'tte_3year_80_20_split', 
    'tte_3year', 
    odsl_feature_list)

In [12]:
imputation_kernel_3year_tte = mf.ImputationKernel(
    X_train_3year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [13]:
imputation_kernel_3year_tte.mice(
    verbose=True,
    iterations=12, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

Initialized logger with name MICE Iterations 1 - 12 and 4 levels
1 Dataset 0
 | HDL | Plasma albumin | Plasma oestradiol | Plasma Vitamin D | Past smoking frequency | PM2.5 exposure | Snoring | Plasma testosterone | HbA1c | Plasma total bilirubin | C-reactive protein | LDL | Plasma urate | Triglycerides | eGFR serum creatinine | Total cholesterol | Diet score | Systemic immune inflammation index | Hearing difficulty (self-reported) | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Education | Vitamin C supplementation | Multivitamin supplementation | Poor oral health | Calcium supplementation | Iron supplementation | Selenium supplementation | Glucosamine supplementation | Urban residence | Normal sleep duration | Daytime sleeping frequency | Body mass index | Ethnicity | Tea intake | Caffeinated coffee drinker | Alcohol intake | Insomnia frequency | Current smoking frequency | Systolic blood pressure | Diastolic blood pressure | Salt added to food |

In [14]:
joblib.dump(imputation_kernel_3year_tte, './data/imputed/imputation_kernel_12iter_IOPremoved3year_tte.pkl')

['./data/imputed/imputation_kernel_12iter_IOPremoved3year_tte.pkl']

### save imputed data

In [ ]:
# run if needed
imputation_kernel_3year_tte = load('./data/imputed/imputation_kernel_12iter_IOPremoved3year_tte.pkl')

In [15]:
X_train_imputed_3year_tte = imputation_kernel_3year_tte.complete_data().reset_index(drop=True)
X_test_imputed_3year_tte = imputation_kernel_3year_tte.impute_new_data(X_test_3year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_IOPremoved3year_tte.pkl')
joblib.dump(X_test_imputed_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_IOPremoved3year_tte.pkl')

joblib.dump(y_train_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_IOPremoved3year_tte.pkl')
joblib.dump(y_test_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_IOPremoved3year_tte.pkl')

['./data/imputed/IOPsubcohort_y_test_IOPremoved3year_tte.pkl']

In [16]:
X_merged_imputed_3year_tte = pd.concat((X_train_imputed_3year_tte, X_test_imputed_3year_tte), ignore_index=True)
y_merged_3year_tte = pd.concat((y_train_3year_tte, y_test_3year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_3year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_IOPremoved3year_tte.pkl')
joblib.dump(y_merged_3year_tte, './data/imputed/IOPsubcohort_y_merged_IOPremoved3year_tte.pkl')

['./data/imputed/IOPsubcohort_y_merged_IOPremoved3year_tte.pkl']

In [17]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_3year_tte)

X_train_scaled_3year_tte = pd.DataFrame(scaler.transform(X_train_imputed_3year_tte), columns=X_train_3year_tte.columns)
X_test_scaled_3year_tte = pd.DataFrame(scaler.transform(X_test_imputed_3year_tte), columns=X_test_3year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_IOPremoved3year_tte.pkl')
joblib.dump(X_test_scaled_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved3year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved3year_tte.pkl']

<br>

<br>

## 5 year

### split into train/test (80/20)

In [18]:
# 1. First, separate cases and controls
cases = merged_df[merged_df['tte_5year'] == 'Glaucoma']
controls = merged_df[merged_df['tte_5year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_5year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_5year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_5year_80_20_split'] = 'train'
test_df['tte_5year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOPremoved_5year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOPremoved_5year_tte['tte_5year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOPremoved_5year_tte['tte_5year_80_20_split'],
    IOPremoved_5year_tte['tte_5year'],
    normalize='index'
))

Final Split Proportions:
tte_5year_80_20_split
train    0.799997
test     0.200003
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_5year               Control  Glaucoma
tte_5year_80_20_split                    
test                   0.995124  0.004876
train                  0.995135  0.004865


In [19]:
IOPremoved_5year_tte.to_csv('IOPremoved_5year_tte.csv', index=False)

In [ ]:
# run if needed
IOPremoved_5year_tte = pd.read_csv('IOPremoved_5year_tte.csv')

### imputation

In [7]:
# run if needed
IOPremoved_5year_tte = pd.read_csv('IOPremoved_5year_tte.csv')

/tmp/ipykernel_3233842/1450914048.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  IOPremoved_5year_tte = pd.read_csv('IOPremoved_5year_tte.csv')


In [8]:
removed_features = ['Corneal hysteresis inter-eye difference', 'Spherical equivalent', 'Corneal hysteresis',
                    'Corneal resistance factor',
                    'Speech reception threshold', 'Tinnitus frequency (self-reported)', 'Arterial stiffness index',
                    'Private healthcare utilisation', 'Exercise (summed MET minutes per week)', 'Total household income']

In [9]:
odsl_feature_list = odsl_feature_list[~np.isin(odsl_feature_list, removed_features)]


odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
merged_df[odsl_categorical_features] = merged_df[odsl_categorical_features].astype('category')

In [10]:
X_train_5year_tte, y_train_5year_tte, X_test_5year_tte, y_test_5year_tte = model_util.get_train_test_datasets(
    IOPremoved_5year_tte, 
    'tte_5year_80_20_split', 
    'tte_5year', 
    odsl_feature_list)

In [11]:
imputation_kernel_5year_tte = mf.ImputationKernel(
    X_train_5year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [12]:
imputation_kernel_5year_tte.mice(
    verbose=True,
    iterations=12, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

Initialized logger with name MICE Iterations 1 - 12 and 4 levels
1 Dataset 0
 | HDL | Plasma albumin | Plasma oestradiol | Plasma Vitamin D | Past smoking frequency | PM2.5 exposure | Snoring | Plasma testosterone | HbA1c | Plasma total bilirubin | C-reactive protein | LDL | Plasma urate | Triglycerides | eGFR serum creatinine | Total cholesterol | Diet score | Systemic immune inflammation index | Hearing difficulty (self-reported) | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Education | Vitamin C supplementation | Multivitamin supplementation | Poor oral health | Calcium supplementation | Iron supplementation | Selenium supplementation | Glucosamine supplementation | Urban residence | Normal sleep duration | Daytime sleeping frequency | Body mass index | Ethnicity | Tea intake | Caffeinated coffee drinker | Alcohol intake | Insomnia frequency | Current smoking frequency | Systolic blood pressure | Diastolic blood pressure | Salt added to food |

In [13]:
joblib.dump(imputation_kernel_5year_tte, './data/imputed/imputation_kernel_12iter_IOPremoved5year_tte.pkl')

['./data/imputed/imputation_kernel_12iter_IOPremoved5year_tte.pkl']

### save imputed data

In [ ]:
# run if needed
imputation_kernel_5year_tte = load('./data/imputed/imputation_kernel_12iter_IOPremoved5year_tte.pkl')

In [14]:
X_train_imputed_5year_tte = imputation_kernel_5year_tte.complete_data().reset_index(drop=True)
X_test_imputed_5year_tte = imputation_kernel_5year_tte.impute_new_data(X_test_5year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_IOPremoved5year_tte.pkl')
joblib.dump(X_test_imputed_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_IOPremoved5year_tte.pkl')

joblib.dump(y_train_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_IOPremoved5year_tte.pkl')
joblib.dump(y_test_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_IOPremoved5year_tte.pkl')

['./data/imputed/IOPsubcohort_y_test_IOPremoved5year_tte.pkl']

In [15]:
X_merged_imputed_5year_tte = pd.concat((X_train_imputed_5year_tte, X_test_imputed_5year_tte), ignore_index=True)
y_merged_5year_tte = pd.concat((y_train_5year_tte, y_test_5year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_5year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_IOPremoved5year_tte.pkl')
joblib.dump(y_merged_5year_tte, './data/imputed/IOPsubcohort_y_merged_IOPremoved5year_tte.pkl')

['./data/imputed/IOPsubcohort_y_merged_IOPremoved5year_tte.pkl']

In [16]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_5year_tte)

X_train_scaled_5year_tte = pd.DataFrame(scaler.transform(X_train_imputed_5year_tte), columns=X_train_5year_tte.columns)
X_test_scaled_5year_tte = pd.DataFrame(scaler.transform(X_test_imputed_5year_tte), columns=X_test_5year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_IOPremoved5year_tte.pkl')
joblib.dump(X_test_scaled_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved5year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved5year_tte.pkl']

<br>

## 10 year

### split into train/test (80/20)

In [4]:
# 1. First, separate cases and controls
cases = merged_df[merged_df['tte_10year'] == 'Glaucoma']
controls = merged_df[merged_df['tte_10year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_10year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_10year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_10year_80_20_split'] = 'train'
test_df['tte_10year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOPremoved_10year_tte = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOPremoved_10year_tte['tte_10year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOPremoved_10year_tte['tte_10year_80_20_split'],
    IOPremoved_10year_tte['tte_10year'],
    normalize='index'
))

Final Split Proportions:
tte_10year_80_20_split
train    0.799998
test     0.200002
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_10year               Control  Glaucoma
tte_10year_80_20_split                    
test                    0.988161  0.011839
train                   0.988166  0.011834


In [5]:
IOPremoved_10year_tte.to_csv('IOPremoved_10year_tte.csv', index=False)

In [ ]:
# run if needed
IOPremoved_10year_tte = pd.read_csv('IOPremoved_10year_tte.csv')

<br>

### imputation

In [4]:
# run if needed
IOPremoved_10year_tte = pd.read_csv('IOPremoved_10year_tte.csv')

/tmp/ipykernel_1727893/677009924.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  IOPremoved_10year_tte = pd.read_csv('IOPremoved_10year_tte.csv')


In [6]:
removed_features = ['Corneal hysteresis inter-eye difference', 'Spherical equivalent', 'Corneal hysteresis',
                    'Corneal resistance factor',
                    'Speech reception threshold', 'Tinnitus frequency (self-reported)', 'Arterial stiffness index',
                    'Private healthcare utilisation', 'Exercise (summed MET minutes per week)', 'Total household income']

In [7]:
odsl_feature_list = odsl_feature_list[~np.isin(odsl_feature_list, removed_features)]


odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
merged_df[odsl_categorical_features] = merged_df[odsl_categorical_features].astype('category')

In [8]:
X_train_10year_tte, y_train_10year_tte, X_test_10year_tte, y_test_10year_tte = model_util.get_train_test_datasets(
    IOPremoved_10year_tte, 
    'tte_10year_80_20_split', 
    'tte_10year', 
    odsl_feature_list)

In [9]:
imputation_kernel_10year_tte = mf.ImputationKernel(
    X_train_10year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending')

In [10]:
imputation_kernel_10year_tte.mice(
    verbose=True,
    iterations=12, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

Initialized logger with name MICE Iterations 1 - 12 and 4 levels
1 Dataset 0
 | HDL | Plasma albumin | Plasma oestradiol | Plasma Vitamin D | Past smoking frequency | PM2.5 exposure | Snoring | Plasma testosterone | HbA1c | Plasma total bilirubin | C-reactive protein | LDL | Plasma urate | Triglycerides | eGFR serum creatinine | Total cholesterol | Diet score | Systemic immune inflammation index | Hearing difficulty (self-reported) | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Education | Vitamin C supplementation | Multivitamin supplementation | Poor oral health | Calcium supplementation | Iron supplementation | Selenium supplementation | Glucosamine supplementation | Urban residence | Normal sleep duration | Daytime sleeping frequency | Body mass index | Ethnicity | Tea intake | Caffeinated coffee drinker | Alcohol intake | Insomnia frequency | Current smoking frequency | Systolic blood pressure | Diastolic blood pressure | Salt added to food |

In [11]:
joblib.dump(imputation_kernel_10year_tte, './data/imputed/imputation_kernel_12iter_IOPremoved10year_tte.pkl')

['./data/imputed/imputation_kernel_12iter_IOPremoved10year_tte.pkl']

<br>

### save imputed data

In [ ]:
# run if needed
imputation_kernel_10year_tte = load('./data/imputed/imputation_kernel_12iter_IOPremoved10year_tte.pkl')

In [12]:
X_train_imputed_10year_tte = imputation_kernel_10year_tte.complete_data().reset_index(drop=True)
X_test_imputed_10year_tte = imputation_kernel_10year_tte.impute_new_data(X_test_10year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_IOPremoved10year_tte.pkl')
joblib.dump(X_test_imputed_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_IOPremoved10year_tte.pkl')

joblib.dump(y_train_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_IOPremoved10year_tte.pkl')
joblib.dump(y_test_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_IOPremoved10year_tte.pkl')

['./data/imputed/IOPsubcohort_y_test_IOPremoved10year_tte.pkl']

In [13]:
X_merged_imputed_10year_tte = pd.concat((X_train_imputed_10year_tte, X_test_imputed_10year_tte), ignore_index=True)
y_merged_10year_tte = pd.concat((y_train_10year_tte, y_test_10year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_10year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_IOPremoved10year_tte.pkl')
joblib.dump(y_merged_10year_tte, './data/imputed/IOPsubcohort_y_merged_IOPremoved10year_tte.pkl')

['./data/imputed/IOPsubcohort_y_merged_IOPremoved10year_tte.pkl']

In [14]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_10year_tte)

X_train_scaled_10year_tte = pd.DataFrame(scaler.transform(X_train_imputed_10year_tte), columns=X_train_10year_tte.columns)
X_test_scaled_10year_tte = pd.DataFrame(scaler.transform(X_test_imputed_10year_tte), columns=X_test_10year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_IOPremoved10year_tte.pkl')
joblib.dump(X_test_scaled_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved10year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_IOPremoved10year_tte.pkl']